Alban Dietrich, UNI: ad4017. 19 February 2023

# Case Study 1

Import libraries.

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import matplotlib.pyplot as plt
import time
import scipy
import scipy.integrate as integrate
from scipy.stats import norm
import math

We define the needed parameters.

In [2]:
# Fixed Parameters
S0 = 1900
K_list = [2000, 2100, 2200]
r = 0.02
q = 0.0187
sig = 0.36
T = 0.25

In [3]:
params = []
params.append(sig)

Let us first calculate the option premiums to have an idea later if our estimations are good or no. Here we use the Black-Scholes formula.

In [4]:
# Black-Merton-Scholes calculator
def BS_d1(S, K, r, q, sigma, tau):
    ''' Computes d1 for the Black-Merton-Scholes formula '''
    d1 = 1.0*(np.log(1.0 * S/K) + (r - q + sigma**2/2) * tau) / (sigma * np.sqrt(tau))
    return d1

def BS_d2(S, K, r, q, sigma, tau):
    ''' Computes d2 for the Black-Merton-Scholes formula '''
    d2 = 1.0*(np.log(1.0 * S/K) + (r - q - sigma**2/2) * tau) / (sigma * np.sqrt(tau))
    return d2

def BS_price(type_option, S, K, r, q, sigma, T, t=0):
    ''' Computes the Black-Merton-Scholes price for a 'call' or 'put' option '''
    tau = T - t
    d1 = BS_d1(S, K, r, q, sigma, tau)
    d2 = BS_d2(S, K, r, q, sigma, tau)
    if type_option == 'call':
        price = S * np.exp(-q * tau) * norm.cdf(d1) - K * np.exp(-r * tau) * norm.cdf(d2)
    elif type_option == 'put':
        price = K * np.exp(-r * tau) * norm.cdf(-d2) - S * np.exp(-q * tau) * norm.cdf(-d1) 
    return price

In [5]:
print("Option Premiums")
print("For K = 2000: ", BS_price("call", S0, 2000, r, q, sig, T))
print("For K = 2100: ", BS_price("call", S0, 2100, r, q, sig, T))
print("For K = 2200: ", BS_price("call", S0, 2200, r, q, sig, T))

Option Premiums
For K = 2000:  95.2466924265716
For K = 2100:  64.83462030513067
For K = 2200:  42.94717532152765


Now let us define the characteristic function of the model. Here we consider the Black-Scholes model.

In [6]:
# Model under consideration
model = 'BS'

In [7]:
def generic_CF(u, params, S0, r, q, T, model):
    
    if(model=='BS'):
        sig = params[0]
        
        phi = np.exp(1j*(np.log(S0)+(r-q-sig**2/2)*T)*u-1/2*sig**2*u**2*T)
    
    else:
        # Other models can be added later, for the moment just the 'BS' model is considered
        print("No model called ",model)

    return phi

## Fast Fourier Transform (FFT)

This code is based on the one provided during the course.

We define the FTT parameters.

In [9]:
# Parameters for FFT

n_FFT_list = [9,11,13,15]

# Step-size
eta = 0.25

# Damping factor
alpha_list = [0.4,1.0,1.4,3.0]

We define the FTT function.

In [10]:
def genericFFT(params, S0, K, r, q, T, alpha, eta, n, model, lda, beta):
    
    N = 2**n
    
    # forming vector x and strikes km for m=1,...,N
    km = np.zeros((N))
    xX = np.zeros((N))
    
    # discount factor
    df = math.exp(-r*T)
    
    nuJ = np.arange(N)*eta
    psi_nuJ = generic_CF(nuJ-(alpha+1)*1j, params, S0, r, q, T, model)/((alpha + 1j*nuJ)*(alpha+1+1j*nuJ))
    
    for j in range(N):  
        km[j] = beta+j*lda
        if j == 0:
            wJ = (eta/2)
        else:
            wJ = eta
            
        xX[j] = np.exp(-1j*beta*nuJ[j])*df*psi_nuJ[j]*wJ
     
    yY = np.fft.fft(xX)
    cT_km = np.zeros((N))  
    for i in range(N):
        multiplier = np.exp(-alpha*km[i])/math.pi
        cT_km[i] = multiplier*np.real(yY[i])
    
    return km, cT_km

Print the results.

In [11]:
print(' ')
print('===================')
print('Model is %s' % model)
print('-------------------')
    
# FFT

print('||| FTT |||')

for K in K_list:
    
    k = math.log(K)

    for n in n_FFT_list:

        N_FFT = 2**n

        # Step-size in log strike space
        lda_FFT = (2*math.pi/N_FFT)/eta


        # Choice of beta
        beta_FTT = np.log(K)-N_FFT*lda_FFT/2

        for alpha in alpha_list:

            print("k = ", K, " AND n = ",n, " AND alpha = ",alpha)
            start_time = time.time()
            km, cT_km = genericFFT(params, S0, K, r, q, T, alpha, eta, n, model, lda_FFT, beta_FTT)

            # Interpolation
            cT_k = np.interp(k, km, cT_km)

            elapsed_time = time.time() - start_time

            print("Option via FFT: for strike %s the option premium is %6.4f" % (K, cT_k))

            print('FFT execution time was %0.7f' % elapsed_time)

 
Model is BS
-------------------
||| FTT |||
k =  2000  AND n =  9  AND alpha =  0.4
Option via FFT: for strike 2000 the option premium is 95.3281
FFT execution time was 0.0022249
k =  2000  AND n =  9  AND alpha =  1.0
Option via FFT: for strike 2000 the option premium is 95.2467
FFT execution time was 0.0017390
k =  2000  AND n =  9  AND alpha =  1.4
Option via FFT: for strike 2000 the option premium is 95.2467
FFT execution time was 0.0019188
k =  2000  AND n =  9  AND alpha =  3.0
Option via FFT: for strike 2000 the option premium is 95.2467
FFT execution time was 0.0019000
k =  2000  AND n =  11  AND alpha =  0.4
Option via FFT: for strike 2000 the option premium is 95.3281
FFT execution time was 0.0066299
k =  2000  AND n =  11  AND alpha =  1.0
Option via FFT: for strike 2000 the option premium is 95.2467
FFT execution time was 0.0066750
k =  2000  AND n =  11  AND alpha =  1.4
Option via FFT: for strike 2000 the option premium is 95.2467
FFT execution time was 0.0065658
k =  2

## Fractional fast Fourier Transform (FrFFT)

This code is based on the one provided during the course.

We define the FrFTT parameters.

In [12]:
# Parameters for FrFTT

n_FrFFT_list = [6,7,8,9]

# Step-size
eta = 0.25

# Damping factor
alpha_list = [0.4,1.0,1.4,3.0]

# Lda
lda_FrFTT = 0.1

We define the FrFTT function.

In [13]:
def genericFrFFT(params, S0, K, r, q, T, alpha, eta, n, model, lda, beta):
    
    N = 2**n
    gamma = eta*lda/(2*math.pi)

    # initialize x, y, z, and cT_km
    km = np.zeros((N))
    x = np.zeros((N))
    y = np.zeros((2*N), dtype=np.complex)
    z = np.zeros((2*N), dtype=np.complex)
    cT_km = np.zeros((N)) 

    # discount factor
    df = math.exp(-r*T)

    # compute x
    nuJ = np.arange(N)*eta
    psi_nuJ = generic_CF(nuJ-(alpha+1)*1j, params, S0, r, q, T, model)/((alpha + 1j*nuJ)*(alpha+1+1j*nuJ))

    for j in range(N):  
        km[j] = beta+j*lda
        if j == 0:
            wJ = (eta/2)
        else:
            wJ = eta
        x[j] = np.exp(-1j*beta*nuJ[j])*df*psi_nuJ[j]*wJ

    # set up y
    for i in range(N):
        y[i] = np.exp(-1j*math.pi*gamma*i**2)*x[i]
    y[N:] = 0

    # set up z
    for i in range(N):
        z[i] = np.exp(1j*math.pi*gamma*i**2)
    z[N:] = z[:N][::-1]

    # compute xi_hat
    xi_hat = np.fft.ifft(np.fft.fft(y) * np.fft.fft(z))

    # compute call prices
    for i in range(N):
        cT_km[i] = np.exp(-alpha*(beta + i*lda))/math.pi * (np.exp(-1j*math.pi*gamma*i**2)*xi_hat[i]).real

    return km, cT_km

We print the results.

In [14]:
print(' ')
print('===================')
print('Model is %s' % model)
print('-------------------')
    
# FrFFT

print('||| FrFTT |||')

for K in K_list:
    
    k = math.log(K)

    for n in n_FrFFT_list:

        N_FrFFT = 2**n


        # Choice of beta
        beta_FrFTT = np.log(K)-N_FrFFT*lda_FrFTT/2

        for alpha in alpha_list:

            print("k = ", K, " AND n = ",n, " AND alpha = ",alpha)
            start_time = time.time()
            km, cT_km = genericFrFFT(params, S0, K, r, q, T, alpha, eta, n, model, lda_FrFTT, beta_FrFTT)

            # Interpolation
            cT_k = np.interp(k, km, cT_km)

            elapsed_time = time.time() - start_time

            print("Option via FrFFT: for strike %s the option premium is %6.4f" % (K, cT_k))

            print('FrFFT execution time was %0.7f' % elapsed_time)

 
Model is BS
-------------------
||| FrFTT |||
k =  2000  AND n =  6  AND alpha =  0.4
Option via FrFFT: for strike 2000 the option premium is 120.7730
FrFFT execution time was 0.0007439
k =  2000  AND n =  6  AND alpha =  1.0
Option via FrFFT: for strike 2000 the option premium is 49.1717
FrFFT execution time was 0.0004468
k =  2000  AND n =  6  AND alpha =  1.4
Option via FrFFT: for strike 2000 the option premium is 47.6224
FrFFT execution time was 0.0004201
k =  2000  AND n =  6  AND alpha =  3.0
Option via FrFFT: for strike 2000 the option premium is 47.1893
FrFFT execution time was 0.0005460
k =  2000  AND n =  7  AND alpha =  0.4
Option via FrFFT: for strike 2000 the option premium is 53.3165
FrFFT execution time was 0.0007198
k =  2000  AND n =  7  AND alpha =  1.0
Option via FrFFT: for strike 2000 the option premium is 47.6260
FrFFT execution time was 0.0007071
k =  2000  AND n =  7  AND alpha =  1.4
Option via FrFFT: for strike 2000 the option premium is 47.6225
FrFFT executi

## Fourier-cosine (COS) method

This code is based on the lecture 2 and the following article: https://mpra.ub.uni-muenchen.de/8914/.

We define the COS parameters.

In [15]:
ab_list = [(-1, 1), (-4, 4), (-8, 8), (-12, 12)]

# As nothing has been asked in the guidelines. We choose n = 9.

N = 2**n

sig = params[0]

n_cos_list = [6,7,8,9]

We define the COS function.

In [16]:
def chi(k, a, b, c, d):
    return integrate.quad(lambda y: np.exp(y)*np.cos(k*math.pi*(y-a)/(b-a)), c, d)[0]
def phi(k, a, b, c, d):
    return integrate.quad(lambda y: np.cos(k*math.pi*(y-a)/(b-a)),c,d)[0]

In [17]:
def COS_Fourier(a,b,S0,r,q,sigma,T,K,n):
    N = 2**n
    
    A = np.zeros(N)
    V = np.zeros(N)
        
    for i in range(N):
        BS_CF = generic_CF(i*math.pi/(b-a), params, S0/K, r, q, T, model)
        A[i] = 2/(b-a)*np.real(BS_CF*np.exp(-1j*i*a*math.pi/(b-a)))
        
        V[i] = 2/(b-a)*K*(chi(i,a,b,0,b)-phi(i,a,b,0,b))
        
    V[0] = 0.5*V[0]
    
    # Here we choose C = e^(-r*T)
    return 0.5*(b-a)*sum(A*V)*math.exp(-r*T)

We calculate the results and print them.

In [18]:
print(' ')
print('===================')
print('Model is %s' % model)
print('-------------------')
    
# COS-Fourier

print("||| COS-Fourier |||")

for K in K_list:
    
    for n in n_cos_list:
    
        for a, b in ab_list:        
                
            start_time = time.time()
            cT_k = COS_Fourier(a,b,S0,r,q,params,T,K,n)
            elapsed_time = time.time() - start_time

            print('COS-Fourier execution time was %0.7f' % elapsed_time)
            print("K = ",K, " AND n = ",n, " AND interval [a, b] = [",a,",",b,'].')

            print("Option via COS-Fourier: for strike %s the option premium is %6.4f" % (K, cT_k))

 
Model is BS
-------------------
||| COS-Fourier |||
COS-Fourier execution time was 0.0295050
K =  2000  AND n =  6  AND interval [a, b] = [ -1 , 1 ].
Option via COS-Fourier: for strike 2000 the option premium is 95.2467
COS-Fourier execution time was 0.0192578
K =  2000  AND n =  6  AND interval [a, b] = [ -4 , 4 ].
Option via COS-Fourier: for strike 2000 the option premium is 95.2475
COS-Fourier execution time was 0.0165839
K =  2000  AND n =  6  AND interval [a, b] = [ -8 , 8 ].
Option via COS-Fourier: for strike 2000 the option premium is 94.9038
COS-Fourier execution time was 0.0144000
K =  2000  AND n =  6  AND interval [a, b] = [ -12 , 12 ].
Option via COS-Fourier: for strike 2000 the option premium is -22626.0999
COS-Fourier execution time was 0.0576713
K =  2000  AND n =  7  AND interval [a, b] = [ -1 , 1 ].
Option via COS-Fourier: for strike 2000 the option premium is 95.2467
COS-Fourier execution time was 0.0604022
K =  2000  AND n =  7  AND interval [a, b] = [ -4 , 4 ].
Op

What happens if we take bigger value of a and b (in absolute value)?

First let us create a new list.

In [19]:
ab_list_sensitivity = [(-16, 16), (-22, 22), (-28, 28), (-34, 34)]

Now let us calculate the outputs.

In [20]:
for K in K_list:
    
    for n in n_cos_list:
    
        for a, b in ab_list_sensitivity:    
                
                start_time = time.time()
                cT_k = COS_Fourier(a,b,S0,r,q,params,T,K,n)
                elapsed_time = time.time() - start_time

                print('COS-Fourier execution time was %0.7f' % elapsed_time)
                print("K = ",K, " AND n = ",n, " AND interval [a, b] = [",a,",",b,'].')

                print("Option via COS-Fourier: for strike %s the option premium is %6.4f" % (K, cT_k))

COS-Fourier execution time was 0.0144742
K =  2000  AND n =  6  AND interval [a, b] = [ -16 , 16 ].
Option via COS-Fourier: for strike 2000 the option premium is -3972177.5854
COS-Fourier execution time was 0.0135472
K =  2000  AND n =  6  AND interval [a, b] = [ -22 , 22 ].
Option via COS-Fourier: for strike 2000 the option premium is -3651513946.3390
COS-Fourier execution time was 0.0133030
K =  2000  AND n =  6  AND interval [a, b] = [ -28 , 28 ].
Option via COS-Fourier: for strike 2000 the option premium is -2304922580373.5674
COS-Fourier execution time was 0.0123420
K =  2000  AND n =  6  AND interval [a, b] = [ -34 , 34 ].
Option via COS-Fourier: for strike 2000 the option premium is -1240503042752099.0000
COS-Fourier execution time was 0.0561810
K =  2000  AND n =  7  AND interval [a, b] = [ -16 , 16 ].
Option via COS-Fourier: for strike 2000 the option premium is 10958.3670
COS-Fourier execution time was 0.0522969
K =  2000  AND n =  7  AND interval [a, b] = [ -22 , 22 ].
Optio

COS-Fourier execution time was 0.6558900
K =  2200  AND n =  9  AND interval [a, b] = [ -34 , 34 ].
Option via COS-Fourier: for strike 2200 the option premium is 594313563.9337


Let us now consider the integrated expression of chi and phi to speed up the calculation. This is based on the article mentioned above.

In [21]:
def chi_integrated(k, a, b, c, d):
    u  = k * np.pi/(b-a)
    chi = 1/(1+u**2)*(np.cos(u * (d-a)) * np.exp(d) - np.cos(u * (c-a)) * np.exp(c) + u*np.sin(u * (d-a)) * np.exp(d)- u*np.sin(u * (c-a)) * np.exp(c))
    return chi


def phi_integrated(k, a, b, c, d):
    if k==0:
        psi = d-c
    else:
        u     = k * np.pi/(b-a)
        psi    = 1/u * ( np.sin(u * (d-a)) - np.sin(u * (c-a)) )

    return psi

In [22]:
def COS_Fourier_integrated(a,b,S0,r,q,sigma,T,K,n):
    N = 2**n
    
    A = np.zeros(N)
    V = np.zeros(N)
        
    for i in range(N):
        BS_CF = generic_CF(i*math.pi/(b-a), params, S0/K, r, q, T, model)
        A[i] = 2/(b-a)*np.real(BS_CF*np.exp(-1j*i*a*math.pi/(b-a)))
        
        V[i] = 2/(b-a)*K*(chi_integrated(i,a,b,0,b)-phi_integrated(i,a,b,0,b))
        
    V[0] = 0.5*V[0]
    
    # Here we choose C = e^(-r*T)
    return 0.5*(b-a)*sum(A*V)*math.exp(-r*T)

In [23]:
print(' ')
print('===================')
print('Model is %s' % model)
print('-------------------')
    
# COS-Fourier

print("||| COS-Fourier-Integrated |||")

for K in K_list:
    
    for n in n_cos_list:
    
        for a, b in ab_list:
        
            start_time = time.time()
            cT_k = COS_Fourier_integrated(a,b,S0,r,q,params,T,K,n)
            elapsed_time = time.time() - start_time

            print('COS-Fourier execution time was %0.7f' % elapsed_time)
            print("K = ",K, " AND n = ",n, " AND interval [a, b] = [",a,",",b,'].')

            print("Option via COS-Fourier: for strike %s the option premium is %6.4f" % (K, cT_k))

 
Model is BS
-------------------
||| COS-Fourier-Integrated |||
COS-Fourier execution time was 0.0005379
K =  2000  AND n =  6  AND interval [a, b] = [ -1 , 1 ].
Option via COS-Fourier: for strike 2000 the option premium is 95.2467
COS-Fourier execution time was 0.0005519
K =  2000  AND n =  6  AND interval [a, b] = [ -4 , 4 ].
Option via COS-Fourier: for strike 2000 the option premium is 95.2475
COS-Fourier execution time was 0.0005040
K =  2000  AND n =  6  AND interval [a, b] = [ -8 , 8 ].
Option via COS-Fourier: for strike 2000 the option premium is 94.9038
COS-Fourier execution time was 0.0005240
K =  2000  AND n =  6  AND interval [a, b] = [ -12 , 12 ].
Option via COS-Fourier: for strike 2000 the option premium is -22626.0999
COS-Fourier execution time was 0.0010159
K =  2000  AND n =  7  AND interval [a, b] = [ -1 , 1 ].
Option via COS-Fourier: for strike 2000 the option premium is 95.2467
COS-Fourier execution time was 0.0010121
K =  2000  AND n =  7  AND interval [a, b] = [ -